In [1]:
import numpy as np

from sklearn.preprocessing import StandardScaler
from torch.utils.tensorboard import SummaryWriter

from tp1 import mse, linear
from tp2 import torch, Housing_data, Net
from torch.utils.data import DataLoader

import torchvision


%load_ext autoreload
%autoreload 2

In [2]:
#mse = MSE()
#linear = Linear()
dataset  = Housing_data()

In [3]:
def SGD(dataloader, parameters, epsilon=1e-3, n_iter=int(1e2), tag='SGD'):
    writer = SummaryWriter(log_dir=f'../log_dir/{tag}')
    losses = []
        
    w, b  = parameters
    
    for epoch in range(n_iter):
        epoch_loss = []
        for X, y in dataloader:
            y_pred = linear(X,w,b)
    
            loss = mse(y_pred, y)
            epoch_loss.append(loss.item())
            loss.backward()
            with torch.no_grad():
                w  -= w.grad*epsilon
                b  -= b.grad*epsilon
            w.grad.zero_()
            b.grad.zero_()
        
        epoch_loss = np.mean(epoch_loss)
        losses.append(epoch_loss)
        writer.add_scalar('Loss/train', epoch_loss, epoch)
        print(f'Epoch {epoch} ==> Loss: {epoch_loss}')
    return losses

In [ ]:
n_feature = 13

w = torch.normal(0, 1/np.sqrt(n_feature), size=(n_feature, 1), requires_grad=True, dtype=torch.float64)
b = torch.randn(1, dtype=torch.float64, requires_grad=True)


BATCH_SIZE = 1
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

losses = SGD(dataloader, (w,b), epsilon=1e-4, n_iter=int(1e3), tag='SGD')

In [ ]:
BATCH_SIZE = 16
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
w = torch.normal(0, 1/np.sqrt(n_feature), size=(n_feature, 1), requires_grad=True, dtype=torch.float64)
b = torch.randn(1, dtype=torch.float64, requires_grad=True)


losses = SGD(dataloader, (w,b), epsilon=1e-3, n_iter=int(1e3), tag='SGD')

In [ ]:
dataloader = DataLoader(dataset, batch_size=len(dataset), shuffle=True)
losses = SGD(dataloader, (w,b), epsilon=1e-3, n_iter=int(1e2), tag='SGD')

In [2]:
path = '../data/mnist'

batch_size_train = 64
batch_size_test = 1000


train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(path, train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(path, train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [7]:
mdl= Net((28*28,49))

In [8]:
EPS = 1e-2
mdl = mdl.train()
optim = torch.optim.Adam(params=mdl.parameters(),lr=EPS) ## on optimise selon w et b, lr : pas de gradient
optim.zero_grad()
criterion = torch.nn.MSELoss()

In [9]:
for epoch in range(10):
    losses = []
    for x,_ in train_loader:
        x = x.reshape(x.shape[0], x.shape[2]*x.shape[3])
        
        xhat = mdl(x)
        
        loss = criterion(xhat, x)
        
        losses.append(loss.item())
        
        loss.backward()
        
        optim.step()           
        optim.zero_grad()
        
    print(f'Loss: {np.mean(losses)} ======> Epoch: {epoch}')

Loss: 0.43491968353674104 ======> Epoch: 0
Loss: 0.4090412420186915 ======> Epoch: 1
Loss: 0.40206656368302385 ======> Epoch: 2
Loss: 0.393080393960481 ======> Epoch: 3
Loss: 0.385962201524645 ======> Epoch: 4
Loss: 0.3848454639601555 ======> Epoch: 5
Loss: 0.3842916318030754 ======> Epoch: 6
Loss: 0.3840862128144897 ======> Epoch: 7
Loss: 0.3841717267976895 ======> Epoch: 8
Loss: 0.38394329962191553 ======> Epoch: 9


In [13]:
for x,_ in test_loader:
    x = x.reshape(x.shape[0], x.shape[2]*x.shape[3])
        
    xhat = mdl(x)
        
    loss = criterion(xhat, x)        
        
    print(f'Loss: {loss.item()} ======> Epoch: {epoch}')

Loss: 0.3828071653842926 ======> Epoch: 9
Loss: 0.38197675347328186 ======> Epoch: 9
Loss: 0.38091859221458435 ======> Epoch: 9
Loss: 0.3836555778980255 ======> Epoch: 9
Loss: 0.37731513381004333 ======> Epoch: 9
Loss: 0.38242045044898987 ======> Epoch: 9
Loss: 0.37346962094306946 ======> Epoch: 9
Loss: 0.38179969787597656 ======> Epoch: 9
Loss: 0.38079744577407837 ======> Epoch: 9
Loss: 0.37647148966789246 ======> Epoch: 9
